In [2]:
from default import PROJECT_HOME
%cd {PROJECT_HOME}

/Users/aflamant/Documents/courses/2024-2025/mémoire/03-code/memoire


In [3]:
from dataset import FixedPrattTrussDataset, FixedPrattTrussDatasetSingleTarget, FixedPrattTrussDatasetThreeTargets
import torch
from torch import nn
from torch.utils.data import DataLoader

In [4]:
def full_target_from_ea(EA: torch.Tensor, n: int = 29) -> torch.Tensor:
    return EA.repeat((1, n))


def full_target_from_ea(EA: torch.Tensor) -> torch.Tensor:
    t = torch.zeros((len(EA), 29))
    t[:, (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 21, 28)] = EA[:, 0].unsqueeze(1)
    t[:, (14, 15, 16, 17, 18, 19, 20)] = EA[:, 1].unsqueeze(1)
    t[:, (22, 23, 24, 25, 26, 27)] = EA[:, 2].unsqueeze(1)

    return t

In [5]:
class EnergyLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, EA, e, q, r, m_incidence):
        n_sample = len(EA)
        residual: torch.Tensor = torch.zeros((n_sample, ds.n_nodes, 2))
        for node_idx, node_incidence in enumerate(m_incidence):
            axial_forces: torch.Tensor = EA * e
            for idx, vect in node_incidence.items():
                residual[:, node_idx] += vect.repeat((n_sample, 1)) * axial_forces[:, idx].unsqueeze(1)
            residual[:, node_idx] -= q[:, node_idx]
            residual[:, node_idx] -= r[:, node_idx]

        return residual.square().mean()


loss = EnergyLoss()

In [6]:
ds = FixedPrattTrussDataset("data/dataset/pratt_truss_bridge_isostatic/1_category_ea/train_128.hdf5")
dl = DataLoader(ds, batch_size=5)
x, y, u, q, w, r = dl.__iter__().__next__()


def compute_oriented_incidence_matrix(ds, n=0):
    n_nodes: int = ds.n_nodes
    n_elements: int = ds.n_elems
    m = torch.zeros((n_nodes, n_elements, 2))
    coordinates = torch.tensor(ds.nodes_coordinate[n].reshape((n_nodes, 2)))
    for ele_idx, (n1_idx, n2_idx) in enumerate(ds.connectivity_matrix[n]):
        n1 = coordinates[n1_idx]
        n2 = coordinates[n2_idx]

        v1 = n1 - n2
        v2 = n2 - n1

        m[n1_idx, ele_idx] = v1 / v1.norm()
        m[n2_idx, ele_idx] = v2 / v2.norm()

    return [{j: v for j, v in enumerate(n_incidence) if not torch.all(v == 0.)} for n_incidence in m]


m_incidence = compute_oriented_incidence_matrix(ds)

In [14]:
strain = ds.bars_strain[0]
E = ds.bars_young[0]
A = ds.bars_area[0]
L = ds.bars_length_init[0]
u = ds.nodes_displacement[0].reshape((-1,2))
q = ds.load[0].reshape((-1,2))
r = ds.support_reaction[0].reshape((-1,2))

In [65]:
U = (E * A / L * (strain*L)**2).sum()
U

W = 0
for i in range(16):
    W += u[i] @ q[i]
    W += u[i] @ r[i]
W

U - W

3.6088749766349792e-09

In [30]:

axial_forces = ds.bars_force[0]

sum_forces = torch.zeros((ds.n_nodes, 2))
for node_idx, node_incidence in enumerate(m_incidence):
    axial_forces = ds.bars_strain[0] * ds.bars_young[0] * ds.bars_area[0]
    q = ds.load[0].reshape((ds.n_nodes, 2))
    r = ds.support_reaction[0].reshape((ds.n_nodes, 2))
    for idx, vect in node_incidence.items():
        sum_forces[node_idx] += vect * axial_forces[idx]
    sum_forces[node_idx] -= q[node_idx]
    sum_forces[node_idx] -= r[node_idx]

print(sum_forces)

tensor([[-1.1642e-08, -1.8299e-02],
        [ 0.0000e+00,  1.8289e-03],
        [ 6.2500e-02, -4.1006e-02],
        [-6.2500e-02,  2.1494e-02],
        [-1.0938e-01,  3.5986e-02],
        [ 1.2500e-01,  2.0173e-03],
        [-2.5000e-01, -7.4493e-02],
        [ 0.0000e+00,  4.4829e-03],
        [ 2.5000e-01, -1.3930e-01],
        [ 0.0000e+00,  2.5095e-01],
        [-1.2500e-01,  2.1494e-02],
        [-1.4062e-01,  5.8694e-03],
        [ 0.0000e+00,  2.7123e-04],
        [ 2.5000e-01, -4.1006e-02],
        [ 6.2500e-02, -4.1006e-02],
        [-6.2500e-02,  6.3450e-02]])


In [24]:
q

array([[      0.        ,       0.        ],
       [      0.        , -916382.75182894],
       [      0.        ,  -38339.3339944 ],
       [      0.        ,  -38339.3339944 ],
       [      0.        , -671300.78598601],
       [      0.        , -686179.68951733],
       [      0.        , -897270.48800708],
       [      0.        , -921870.1919829 ],
       [      0.        ,       0.        ],
       [      0.        ,  -41588.00094965],
       [      0.        ,  -38339.3339944 ],
       [      0.        ,  -38339.3339944 ],
       [      0.        ,  -19497.61941186],
       [      0.        ,  -38339.3339944 ],
       [      0.        ,  -38339.3339944 ],
       [      0.        ,  -41588.00094965]])